In [1]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.callbacks import LearningRateScheduler
from keras.regularizers import l2

c:\users\kumar\anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [3]:
# Hyperparameters
batch_size = 32
num_classes = 10
epochs = 50
l = 12
num_filter = 40
compression = 1
dropout_rate = 0.2
weight_decay=1E-4

In [4]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

#Dividing pixel values by 255

x_train = x_train/255
x_test = x_test/255
# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [5]:
print(x_train.shape,y_train.shape)

(50000, 32, 32, 3) (50000, 10)


In [6]:
# Dense Block
def add_denseblock(input, num_filter = 40, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])        
        temp = concat
        
    return temp

In [7]:
def add_transition(input, num_filter = 40, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [8]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [9]:
dropout_rate = 0.2
l = 12
num_filter=40
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same', kernel_initializer='he_uniform')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block,num_filter , dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

#Fourth_Block = add_denseblock(Third_Transition, num_filter, dropout_rate)
#Fourth_Transition = add_transition(Fourth_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [10]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 40)   1080        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 40)   160         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 40)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [11]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])
filepath="weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [12]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=50,
                    verbose=1,
                    validation_data=(x_test, y_test),callbacks=callbacks_list)

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 508s 10ms/step - loss: 1.3827 - acc: 0.5038 - val_loss: 2.4520 - val_acc: 0.4063

Epoch 00001: val_acc improved from -inf to 0.40630, saving model to weights.best.hdf5
Epoch 2/50
50000/50000 [==============================] - 512s 10ms/step - loss: 0.8631 - acc: 0.6960 - val_loss: 1.2305 - val_acc: 0.6469

Epoch 00002: val_acc improved from 0.40630 to 0.64690, saving model to weights.best.hdf5
Epoch 3/50
50000/50000 [==============================] - 513s 10ms/step - loss: 0.6887 - acc: 0.7588 - val_loss: 0.9923 - val_acc: 0.7185

Epoch 00003: val_acc improved from 0.64690 to 0.71850, saving model to weights.best.hdf5
Epoch 4/50
50000/50000 [==============================] - 511s 10ms/step - loss: 0.5825 - acc: 0.7974 - val_loss: 1.1107 - val_acc: 0.7000

Epoch 00004: val_acc did not improve
Epoch 5/50
50000/50000 [==============================] - 511s 10ms/step - loss: 0.5037 -

In [13]:
model.load_weights("weights.best.hdf5")
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 33s 3ms/step
Test loss: 0.5171954084873199
Test accuracy: 0.8963
